In [17]:
class A:
    
    @classmethod
    def rand_shit(cls):
        cls.print_1()
        cls.print_2()

    @staticmethod
    def print_1():
        print("hehe")

    @staticmethod
    def print_2():
        print("hoho")

    
        

In [18]:
A.rand_shit()

hehe
hoho
